# Deploy Charmed Kubeflow on Charmed Kubernetes cluster on AWS

Contents:


## Setup AWS cloud

Follow [this guide](https://juju.is/docs/olm/amazon-aws) to setup AWS cloud and Juju controller.

## Deploy Charmed Kubernetes cluster on AWS

Checkout Charmed Kubernetes bundle release 1.24 and update cpu, disk, and memory constraints to satisfy Kubeflow requirements:

In [ ]:
git clone https://github.com/charmed-kubernetes/bundle.git
sed -i '/^ *charm: kubernetes-worker/,/^ *[^:]*:/s/constraints: cores=2 mem=8G root-disk=16G/constraints: cores=8 mem=32G root-disk=100G/' ./bundle/releases/1.24/bundle.yaml

Deploy updated Charmed Kubernetes bundle on AWS with storage overlay:

In [ ]:
juju deploy ./bundle/releases/1.24/bundle.yaml --overlay ./bundle/overlays/aws-storage-overlay.yaml --trust
juju scp kubernetes-control-plane/0:config ~/.kube/config
juju add-k8s charmed-k8s-aws --controller $(juju switch | cut -d: -f1) --storage=cdk-ebs

## Deploy Charmed Kubeflow on created Charmed Kubernetes cluster on AWS

Add `kubeflow` model to the controller and deploy Charmed Kubeflow version 1.7:

In [ ]:
juju add-model kubeflow charmed-k8s-aws
juju deploy kubeflow --channel 1.7/stable --trust